# Clasificación de tokens con HuggingFace

In [1]:
import sys
sys.path.append('../datos/procesamiento')
from corpus import Corpus

corpus_train = Corpus()
corpus_test = Corpus()

---

## Cargando el corpus como dataset de HuggingFace

In [2]:
ner_dict = {'O': 0,
            'B-ACTIVE_PRINCIPLE': 1,
            'I-ACTIVE_PRINCIPLE': 2,
            'B-FORMA_FARMA':3,
            'I-FORMA_FARMA':4,
            'B-ADMIN': 5,
            'I-ADMIN': 6,
            'B-PERIODICITY': 7,
            'I-PERIODICITY': 8,
            'B-DURATION': 9,
            'I-DURATION': 10
            }

corpus_train.entidades = ner_dict
corpus_test.entidades = ner_dict

In [3]:
corpus_train.load_conll('../datos/Etiquetado/corpus_ER_train.txt')

Agregadas 85246 secuencias de token-entidad al corpus


In [4]:
corpus_train.load_conll('../datos/Etiquetado/corpus_ER_test.txt')

Agregadas 51183 secuencias de token-entidad al corpus


In [5]:
from datasets import DatasetDict

HF_dataset = DatasetDict()

HF_dataset['train'] = corpus_train.to_HF_dataset()
HF_dataset['test'] = corpus_test.to_HF_dataset()

In [6]:
HF_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 136429
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 136429
    })
})

## Cargamos un modelo basado en Transformers de HuggingFace

Nuestro modelo será: plncmm/bert-clinical-scratch-wl-es. Ha sido fine-tuneado con texto médico (aunque probablemente no con prescripciones)

[Fuente de esta sección](https://huggingface.co/docs/transformers/tasks/token_classification)

### Tokenizer

In [9]:
MODEL = "plncmm/bert-clinical-scratch-wl-es"

In [10]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL)

Usaremos el tokenizador para codificar nuestro input.

In [11]:
example = HF_dataset["train"][0]

tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

['[CLS]',
 'flu',
 '##cona',
 '##zo',
 '##l',
 '150',
 'mg',
 'cápsula',
 '1',
 'cápsula',
 'oral',
 'cada',
 '3',
 'dias',
 '[SEP]']

Debemos corregir el _mismatch_ entre input tokenizado y la lista de tokens

In [7]:
# from auxfunctions import tokenize_and_align_labels

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [12]:
tokenized_data = HF_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_data = tokenized_data.remove_columns(['id','tokens','ner_tags'])

  0%|          | 0/137 [00:00<?, ?ba/s]

  0%|          | 0/137 [00:00<?, ?ba/s]

In [13]:
tokenized_data.data

{'train': InMemoryTable
 input_ids: list<item: int32>
   child 0, item: int32
 token_type_ids: list<item: int8>
   child 0, item: int8
 attention_mask: list<item: int8>
   child 0, item: int8
 labels: list<item: int64>
   child 0, item: int64
 ----
 input_ids: [[[4,5121,30534,1600,30962,...,12791,1748,1306,12873,5],[4,11236,1348,29117,3284,...,1139,28376,30967,30972,5],...,[4,11247,13436,15275,30956,...,2581,29258,12791,15359,5],[4,8753,3283,2861,22396,...,1411,2242,1074,1707,5]],[[4,15023,13394,1177,1114,...,2596,1672,1129,12873,5],[4,5970,4358,1167,1600,...,1707,1672,1129,12873,5],...,[4,11236,1348,29117,3284,...,29258,12791,1074,23938,5],[4,27625,11425,30960,6821,...,1139,1992,1482,30961,5]],...,[[4,4667,27921,19257,15023,...,1411,2242,1074,1707,5],[4,15721,13067,10791,997,...,2596,1672,1413,12873,5],...,[4,1254,1092,1104,10110,...,2596,1672,1306,2817,5],[4,16768,1307,1275,1316,...,989,1129,1139,1098,5]],[[4,9969,13908,1519,30967,...,15126,2155,9934,2940,5],[4,13695,13456,16387,1527

### Modelo

In [15]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(MODEL, num_labels=len(ner_dict))

Some weights of the model checkpoint at plncmm/bert-clinical-scratch-wl-es were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at plncmm/bert-clinical-s

## Entrenamiento

In [16]:
training_args = TrainingArguments(
    output_dir = "./results",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 5,
    weight_decay = 0.01,
)

In [17]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = tokenized_data["train"],
    eval_dataset = tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator = data_collator,
)

In [18]:
trainer.train()

/home/camilo/miniconda3/envs/entidades_env/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 136429
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 42635


  0%|          | 0/42635 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json


{'loss': 0.2023, 'learning_rate': 1.9765450920605137e-05, 'epoch': 0.06}


Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json


...

Model weights saved in ./results/checkpoint-42500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-42500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-42500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 136429
  Batch size = 16


  0%|          | 0/8527 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.00019751596846617758, 'eval_runtime': 437.8101, 'eval_samples_per_second': 311.617, 'eval_steps_per_second': 19.476, 'epoch': 5.0}
{'train_runtime': 10561.2495, 'train_samples_per_second': 64.589, 'train_steps_per_second': 4.037, 'train_loss': 0.006245762737205974, 'epoch': 5.0}


TrainOutput(global_step=42635, training_loss=0.006245762737205974, metrics={'train_runtime': 10561.2495, 'train_samples_per_second': 64.589, 'train_steps_per_second': 4.037, 'train_loss': 0.006245762737205974, 'epoch': 5.0})

In [19]:
trainer.save_model("bert-clinical-scratch-wl-es-NER-prescription")

Saving model checkpoint to bert-clinical-scratch-wl-es-NER-prescription
Configuration saved in bert-clinical-scratch-wl-es-NER-prescription/config.json
Model weights saved in bert-clinical-scratch-wl-es-NER-prescription/pytorch_model.bin
tokenizer config file saved in bert-clinical-scratch-wl-es-NER-prescription/tokenizer_config.json
Special tokens file saved in bert-clinical-scratch-wl-es-NER-prescription/special_tokens_map.json


## Evaluación de resultados

In [14]:
from transformers import AutoTokenizer
from auxfunctions import tokenize_and_align_labels

MODEL = "plncmm/bert-clinical-scratch-wl-es"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [15]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("bert-clinical-scratch-wl-es-NER-prescription")

In [16]:
from auxfunctions import eval_text, map_entities, calculate_metrics

In [17]:
text = "PARACETAMOL 500 MG COMPRIMIDO 1 COMPRIMIDO ORAL cada 6 horas durante 3 dias"

eval_text(text,tokenizer,model)

array([ 1,  0,  0,  3,  0,  0,  5,  7,  8,  8,  9, 10, 10])

In [18]:
ner_dict = {'O': 0,
            'B-ACTIVE_PRINCIPLE': 1,
            'I-ACTIVE_PRINCIPLE': 2,
            'B-FORMA_FARMA':3,
            'I-FORMA_FARMA':4,
            'B-ADMIN': 5,
            'I-ADMIN': 6,
            'B-PERIODICITY': 7,
            'I-PERIODICITY': 8,
            'B-DURATION': 9,
            'I-DURATION': 10
            }

In [19]:
print(text)
map_entities(eval_text(text,tokenizer,model),ner_dict)

PARACETAMOL 500 MG COMPRIMIDO 1 COMPRIMIDO ORAL cada 6 horas durante 3 dias


['B-ACTIVE_PRINCIPLE',
 'O',
 'O',
 'B-FORMA_FARMA',
 'O',
 'O',
 'B-ADMIN',
 'B-PERIODICITY',
 'I-PERIODICITY',
 'I-PERIODICITY',
 'B-DURATION',
 'I-DURATION',
 'I-DURATION']

Evaluación con otras métricas

In [20]:
y_test = [row['ner_tags'] for row in HF_dataset['test']]

In [24]:
y_preds = []
length = len(HF_dataset['test'])
for i, row in enumerate(HF_dataset['test']):
    if 100*(i+1)/length % 10 == 0:
        print("progreso = {}%".format(100*(i+1)/length))
    y_preds.append(list(eval_text(row['tokens'],tokenizer,model)))

KeyboardInterrupt: 

In [ ]:
print("Métricas en test-etiquetado con ER")
calculate_metrics(y_preds,y_test,ner_dict=ner_dict)

In [26]:
datos_conll = Corpus()
datos_conll.entidades = ner_dict

for i in range(4):
    datos_conll.load_conll('../datos/Etiquetado/corpus_s{}_etiquetados.conll'.format(i+1))

HF_data_mini = datos_conll.to_HF_dataset()
HF_dataset_mini = HF_data_mini.train_test_split(test_size=0.5,seed=0)

Agregadas 250 secuencias de token-entidad al corpus
Agregadas 251 secuencias de token-entidad al corpus
Agregadas 251 secuencias de token-entidad al corpus
Agregadas 251 secuencias de token-entidad al corpus


In [27]:
from auxfunctions import eval_text, map_entities, calculate_metrics

y_test = [row['ner_tags'] for row in HF_dataset_mini['test']]
y_preds = [list(eval_text(row['tokens'],tokenizer,model)) for row in HF_dataset_mini['test']]

print("Métricas en test etiquetado manual")
calculate_metrics(y_preds,y_test,ner_dict=ner_dict)

## Versión mini

A continuación definimos un modelo como el anterior, con la diferencia de que este será entrenado (fine-tunning) en 80% de los datos etiquetados a mano y testeado con 20 % de estos.

In [1]:
import sys
sys.path.append('../datos/procesamiento')
from corpus import Corpus

datos_conll = Corpus()

In [2]:
ner_dict = {'O': 0,
            'B-ACTIVE_PRINCIPLE': 1,
            'I-ACTIVE_PRINCIPLE': 2,
            'B-FORMA_FARMA':3,
            'I-FORMA_FARMA':4,
            'B-ADMIN': 5,
            'I-ADMIN': 6,
            'B-PERIODICITY': 7,
            'I-PERIODICITY': 8,
            'B-DURATION': 9,
            'I-DURATION': 10
            }

datos_conll.entidades = ner_dict

In [3]:
for i in range(4):
    datos_conll.load_conll('../datos/Etiquetado/corpus_s{}_etiquetados.conll'.format(i+1))

Agregadas 250 secuencias de token-entidad al corpus
Agregadas 251 secuencias de token-entidad al corpus
Agregadas 251 secuencias de token-entidad al corpus
Agregadas 251 secuencias de token-entidad al corpus


In [4]:
HF_data_mini = datos_conll.to_HF_dataset()

HF_dataset = HF_data_mini.train_test_split(test_size=0.5,seed=0)
HF_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 501
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 502
    })
})

In [5]:
from transformers import AutoTokenizer
from auxfunctions import tokenize_and_align_labels

MODEL = "plncmm/bert-clinical-scratch-wl-es"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

process = lambda examples: tokenize_and_align_labels(examples,tokenizer)

tokenized_data_mini = HF_dataset.map(process, batched=True)
tokenized_data_mini = tokenized_data_mini.remove_columns(['id','tokens','ner_tags'])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [6]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model_mini = AutoModelForTokenClassification.from_pretrained(MODEL, num_labels=len(ner_dict))

Some weights of the model checkpoint at plncmm/bert-clinical-scratch-wl-es were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at plncmm/bert-clinical-s

In [7]:
training_args = TrainingArguments(
    output_dir = "./results",
    evaluation_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 5,
    weight_decay = 0.01,
)

In [8]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = Trainer(
    model=model_mini,
    args=training_args,
    train_dataset = tokenized_data_mini["train"],
    eval_dataset = tokenized_data_mini["test"],
    tokenizer=tokenizer,
    data_collator = data_collator,
)

In [9]:
trainer.train()

/home/camilo/miniconda3/envs/entidades_env/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 501
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 160


  0%|          | 0/160 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
***** Running Evaluation *****
  Num examples = 502
  Batch size = 16


  0%|          | 0/32 [00:00<?, ?it/s]

{'eval_loss': 0.7695652842521667, 'eval_runtime': 5.6407, 'eval_samples_per_second': 88.996, 'eval_steps_per_second': 5.673, 'epoch': 1.0}


...

***** Running Evaluation *****
  Num examples = 502
  Batch size = 16


  0%|          | 0/32 [00:00<?, ?it/s]



Training completed. Do not forget to share your model on huggingface.co/models =)




{'eval_loss': 0.44520142674446106, 'eval_runtime': 1.4058, 'eval_samples_per_second': 357.08, 'eval_steps_per_second': 22.762, 'epoch': 5.0}
{'train_runtime': 210.5847, 'train_samples_per_second': 11.895, 'train_steps_per_second': 0.76, 'train_loss': 0.6563064575195312, 'epoch': 5.0}


TrainOutput(global_step=160, training_loss=0.6563064575195312, metrics={'train_runtime': 210.5847, 'train_samples_per_second': 11.895, 'train_steps_per_second': 0.76, 'train_loss': 0.6563064575195312, 'epoch': 5.0})

In [10]:
trainer.save_model("bert-clinical-scratch-wl-es-NER-prescription-mini")

Saving model checkpoint to bert-clinical-scratch-wl-es-NER-prescription-mini
Configuration saved in bert-clinical-scratch-wl-es-NER-prescription-mini/config.json
Model weights saved in bert-clinical-scratch-wl-es-NER-prescription-mini/pytorch_model.bin
tokenizer config file saved in bert-clinical-scratch-wl-es-NER-prescription-mini/tokenizer_config.json
Special tokens file saved in bert-clinical-scratch-wl-es-NER-prescription-mini/special_tokens_map.json


In [11]:
model_mini = AutoModelForTokenClassification.from_pretrained("bert-clinical-scratch-wl-es-NER-prescription-mini")

loading configuration file bert-clinical-scratch-wl-es-NER-prescription-mini/config.json
Model config BertConfig {
  "_name_or_path": "bert-clinical-scratch-wl-es-NER-prescription-mini",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings"

In [12]:
from auxfunctions import eval_text, map_entities, calculate_metrics

y_test = [row['ner_tags'] for row in HF_dataset['test']]
y_preds = [list(eval_text(row['tokens'],tokenizer,model_mini)) for row in HF_dataset['test']]

calculate_metrics(y_preds,y_test,ner_dict=ner_dict)

Resultados de evaluación
	 f1: 0.62 | precision: 0.53 | recall: 0.74


(0.5306397306397307, 0.7367928938756428, 0.6169504795458994)